# Day 16

In [1]:
from heapq import heappop, heappush

# Define directions
DIRECTIONS = ['E', 'S', 'W', 'N']  # Clockwise order: East, South, West, North
DELTA = {
    'E': (0, 1),  # East: move right
    'S': (1, 0),  # South: move down
    'W': (0, -1), # West: move left
    'N': (-1, 0)  # North: move up
}

def parse_maze(maze):
    start = end = None
    grid = []
    for r, line in enumerate(maze.strip().split('\n')):
        grid.append(line)
        if 'S' in line:
            start = (r, line.index('S'))
        if 'E' in line:
            end = (r, line.index('E'))
    return grid, start, end

def heuristic(pos, end):
    """Manhattan distance heuristic."""
    return abs(pos[0] - end[0]) + abs(pos[1] - end[1])

def reindeer_maze(maze):
    grid, start, end = parse_maze(maze)
    rows, cols = len(grid), len(grid[0])
    
    # Priority queue for A* (score, position, direction)
    pq = []
    heappush(pq, (0, start, 'E'))  # Start facing East with score 0
    
    # Visited states (position, direction) -> lowest score
    visited = {}
    visited[(start, 'E')] = 0

    while pq:
        score, (x, y), facing = heappop(pq)

        # If we reach the end, return the score
        if (x, y) == end:
            return score

        # Try moving forward
        dx, dy = DELTA[facing]
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] != '#':
            new_score = score + 1
            if (nx, ny, facing) not in visited or new_score < visited[(nx, ny, facing)]:
                visited[(nx, ny, facing)] = new_score
                heappush(pq, (new_score, (nx, ny), facing))

        # Try rotating left or right
        for rotation in [-1, 1]:  # -1 for left, 1 for right
            new_facing = DIRECTIONS[(DIRECTIONS.index(facing) + rotation) % 4]
            new_score = score + 1000
            if (x, y, new_facing) not in visited or new_score < visited[(x, y, new_facing)]:
                visited[(x, y, new_facing)] = new_score
                heappush(pq, (new_score, (x, y), new_facing))

# Example usage
maze1 = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""
maze2 = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""

print(reindeer_maze(maze1))  # Output: 7036
print(reindeer_maze(maze2))  # Output: 11048

with open("Input/InputDay16P1.txt", "r") as f:
    Inputdata = f.read()

print(reindeer_maze(Inputdata))


7036
11048
75416


In [3]:
from collections import deque
from heapq import heappop, heappush

DIRECTIONS = ['E', 'S', 'W', 'N']  # Clockwise: East, South, West, North
DELTA = {
    'E': (0, 1),  # East: move right
    'S': (1, 0),  # South: move down
    'W': (0, -1), # West: move left
    'N': (-1, 0)  # North: move up
}

def parse_maze(maze):
    start = end = None
    grid = []
    for r, line in enumerate(maze.strip().split('\n')):
        grid.append(line)
        if 'S' in line:
            start = (r, line.index('S'))
        if 'E' in line:
            end = (r, line.index('E'))
    return grid, start, end

def heuristic(pos, end):
    """Manhattan distance heuristic."""
    return abs(pos[0] - end[0]) + abs(pos[1] - end[1])

def find_best_score(maze):
    """Find the best score from S to E using A*."""
    grid, start, end = parse_maze(maze)
    rows, cols = len(grid), len(grid[0])
    
    # Priority queue for A* (score, position, direction)
    pq = []
    heappush(pq, (0, start, 'E'))  # Start facing East with score 0
    
    # Visited states (position, direction) -> lowest score
    visited = {}
    visited[(start, 'E')] = 0

    while pq:
        score, (x, y), facing = heappop(pq)

        # If we reach the end, return the score
        if (x, y) == end:
            return score

        # Try moving forward
        dx, dy = DELTA[facing]
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] != '#':
            new_score = score + 1
            if (nx, ny, facing) not in visited or new_score < visited[(nx, ny, facing)]:
                visited[(nx, ny, facing)] = new_score
                heappush(pq, (new_score, (nx, ny), facing))

        # Try rotating left or right
        for rotation in [-1, 1]:  # -1 for left, 1 for right
            new_facing = DIRECTIONS[(DIRECTIONS.index(facing) + rotation) % 4]
            new_score = score + 1000
            if (x, y, new_facing) not in visited or new_score < visited[(x, y, new_facing)]:
                visited[(x, y, new_facing)] = new_score
                heappush(pq, (new_score, (x, y), new_facing))

def find_best_path_tiles(maze):
    """Find all tiles that are part of any best path."""
    grid, start, end = parse_maze(maze)
    rows, cols = len(grid), len(grid[0])
    
    # Get the best score
    best_score = find_best_score(maze)
    
    # Perform BFS to backtrack from end to start
    queue = deque([(end, 0)])  # (position, score)
    visited = set()
    visited.add(end)
    
    best_path_tiles = set()
    best_path_tiles.add(end)

    while queue:
        (x, y), score = queue.popleft()

        # Explore all neighbors (reverse of the forward movement and rotation logic)
        for direction, (dx, dy) in DELTA.items():
            nx, ny = x - dx, y - dy  # Reverse movement
            if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] != '#' and (nx, ny) not in visited:
                new_score = score + 1
                if new_score <= best_score:  # Only explore valid paths
                    queue.append(((nx, ny), new_score))
                    visited.add((nx, ny))
                    best_path_tiles.add((nx, ny))

    return len(best_path_tiles)

# Example usage
maze1 = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""
maze2 = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""

print(find_best_path_tiles(maze1))  # Should match problem's example: 45
print(find_best_path_tiles(maze2))  # Should match problem's example: 64


104
132


In [8]:
import heapq

def solve():
    with open("Input/InputDay16P1.txt", "r") as f:
        grid = [list(line.strip()) for line in f]

    start_row, start_col = -1, -1
    end_row, end_col = -1, -1

    for r in range(len(grid)):
        for c in range(len(grid[0])):
            if grid[r][c] == 'S':
                start_row, start_col = r, c
            elif grid[r][c] == 'E':
                end_row, end_col = r, c

    q = [(0, start_row, start_col, 0)]  # (cost, row, col, direction)
    visited = set()

    while q:
        cost, row, col, direction = heapq.heappop(q)

        if (row, col, direction) in visited:
            continue
        visited.add((row, col, direction))

        if row == end_row and col == end_col:
            return cost

        # Move forward
        dr, dc = [(0, 1), (1, 0), (0, -1), (-1, 0)][direction]
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < len(grid) and 0 <= new_col < len(grid[0]) and grid[new_row][new_col] != '#':
            heapq.heappush(q, (cost + 1, new_row, new_col, direction))

        # Turn clockwise
        new_direction = (direction + 1) % 4
        heapq.heappush(q, (cost + 1000, row, col, new_direction))

        # Turn counterclockwise
        new_direction = (direction - 1 + 4) % 4
        heapq.heappush(q, (cost + 1000, row, col, new_direction))

    return -1

if __name__ == "__main__":
    result = solve()
    print(result)

75416


In [12]:
import heapq

def solve():
    with open("Input/InputDay16P1.txt", "r") as f:
        grid = [list(line.strip()) for line in f]

    rows = len(grid)
    cols = len(grid[0])

    start_row, start_col = -1, -1
    end_row, end_col = -1, -1

    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 'S':
                start_row, start_col = r, c
            elif grid[r][c] == 'E':
                end_row, end_col = r, c

    def is_valid(r, c):
        return 0 <= r < rows and 0 <= c < cols and grid[r][c] != '#'

    def get_neighbors(r, c, direction):
        neighbors = []
        dr, dc = [(0, 1), (1, 0), (0, -1), (-1, 0)][direction]
        if is_valid(r + dr, c + dc):
            neighbors.append(((r + dr, c + dc), direction, 1))  # Move forward

        for new_dir in [(direction + 1) % 4, (direction - 1) % 4]:
            neighbors.append(((r, c), new_dir, 1000)) #Rotate
        return neighbors
    
    def dijkstra():
        q = [(0, start_row, start_col, 0, [(start_row, start_col, 0)])] # score, row, col, direction, path (STARTING WITH START AND DIRECTION)
        visited = set()
        min_cost = float('inf')
        best_paths = []

        while q:
            cost, r, c, direction, path = heapq.heappop(q)

            if (r, c, direction) in visited:
                continue
            visited.add((r, c, direction))

            if r == end_row and c == end_col:
                if cost < min_cost:
                  min_cost = cost
                  best_paths = [path]
                elif cost == min_cost:
                  best_paths.append(path)
                continue

            for (nr, nc), ndir, move_cost in get_neighbors(r, c, direction):
                new_cost = cost + move_cost
                new_path = path[:] # Create a copy!
                if move_cost == 1: # Only add new tile if we moved
                    new_path.append((nr, nc, ndir))
                else:
                    new_path[-1] = (r,c,ndir) #Update the direction for the current tile
                heapq.heappush(q, (new_cost, nr, nc, ndir, new_path))
        return min_cost, best_paths

    min_cost, best_paths = dijkstra()
    
    path_tiles = set()
    for path in best_paths:
        for tile in path:
            path_tiles.add((tile[0], tile[1])) #Only add the row and column

    print(len(path_tiles))


solve()

417


# Part 2

In [18]:
import heapq

def solve(grid):
    rows = len(grid)
    cols = len(grid[0])

    start_row, start_col = -1, -1
    end_row, end_col = -1, -1

    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 'S':
                start_row, start_col = r, c
            elif grid[r][c] == 'E':
                end_row, end_col = r, c

    def is_valid(r, c):
        return 0 <= r < rows and 0 <= c < cols and grid[r][c] != '#'

    def get_neighbors(r, c, direction):
        neighbors = []
        dr, dc = [(0, 1), (1, 0), (0, -1), (-1, 0)][direction]
        if is_valid(r + dr, c + dc):
            neighbors.append(((r + dr, c + dc), direction, 1))  # Move forward

        for new_dir in [(direction + 1) % 4, (direction - 1) % 4]:
            neighbors.append(((r, c), new_dir, 1000)) #Rotate
        return neighbors
    
    def dijkstra():
        q = [(0, start_row, start_col, 0, [(start_row, start_col)])] 
        visited = {}  # Initialize visited as a dictionary!
        min_cost = float('inf')
        best_paths = []

        while q:
            cost, r, c, direction, path = heapq.heappop(q)

            if (r, c, direction) in visited and cost > visited[(r,c,direction)]:
                continue
            visited[(r, c, direction)] = cost

            if r == end_row and c == end_col:
                if cost < min_cost:
                  min_cost = cost
                  best_paths = [path]
                elif cost == min_cost:
                  best_paths.append(path)
                continue

            for (nr, nc), ndir, move_cost in get_neighbors(r, c, direction):
                new_cost = cost + move_cost
                new_path = path[:]
                if move_cost == 1:
                    new_path.append((nr, nc))
                heapq.heappush(q, (new_cost, nr, nc, ndir, new_path))
        return min_cost, best_paths

    min_cost, best_paths = dijkstra()
    
    path_tiles = set()
    for path in best_paths:
        for tile in path:
            path_tiles.add(tile)

    return len(path_tiles)

# Example inputs (same as before)
example1 = [
    list("###############"),
    list("#.......#....E#"),
    list("#.#.###.#.###.#"),
    list("#.....#.#...#.#"),
    list("#.###.#####.#.#"),
    list("#.#.#.......#.#"),
    list("#.#.#####.###.#"),
    list("#...........#.#"),
    list("###.#.#####.#.#"),
    list("#...#.....#.#.#"),
    list("#.#.#.###.#.#.#"),
    list("#.....#...#.#.#"),
    list("#.###.#.#.#.#.#"),
    list("#S..#.....#...#"),
    list("###############"),
]

example2 = [
    list("#################"),
    list("#...#...#...#..E#"),
    list("#.#.#.#.#.#.#.#.#"),
    list("#.#.#.#...#...#.#"),
    list("#.#.#.#.###.#.#.#"),
    list("#...#.#.#.....#.#"),
    list("#.#.#.#.#.#####.#"),
    list("#.#...#.#.#.....#"),
    list("#.#.#####.#.###.#"),
    list("#.#.#.......#...#"),
    list("#.#.###.#####.###"),
    list("#.#.#...#.....#.#"),
    list("#.#.#.#####.###.#"),
    list("#.#.#.........#.#"),
    list("#.#.#.#########.#"),
    list("#S#.............#"),
    list("#################"),
]

print(solve(example1))  # Output: 45
print(solve(example2))  # Output: 64

with open("Input/InputDay16P1.txt", "r") as f:
    grid = [list(line.strip()) for line in f]
print(solve(grid))



45
64
476
